In [1]:
import pandas as pd
import re
import operator
import string
import nltk 
from nltk.tag import pos_tag

In [2]:
gg = pd.read_json('gg2013.json', orient='records')
gg1 = pd.read_json('gg2015.json', orient='records')

In [10]:
copy = gg.copy(deep=True)

In [11]:
def removePunc(x):
#     new_string = x.translate(str.maketrans('', '', string.punctuation))
    x = re.sub(r'[@#]\w+', '', x) #taking out hashtags and @ 
    x = re.sub(r'(https?:\/\/)?([\da-z\.-]+)\.([a-z\.]{2,6})([\/\w \.-]*)', '', x) #taking out links 
    x = re.sub(r'[!?\.,\'\"]+', '', x) #taking out punctuation i.e ? ! . ' and " 
    x = re.sub(r'(g|G)olden (g|G)lobes', '', x) #took out golden globes as a stop word here 
    return x.strip()
# copy['text'] = copy['text'].str.lower()
copy['text'] = copy['text'].apply(lambda x: removePunc(str(x)))
copy.head()

,text,user,id,timestamp_ms
0,JLos dress,"{'screen_name': 'Dozaaa_xo', 'id': 557374298}",290620657987887104,2013-01-14 00:45:38
1,Whats making Sofia Vergaras boobs stay like th...,"{'screen_name': 'theAmberShow', 'id': 14648726}",290620657887219713,2013-01-14 00:45:38
2,RT : Kerry Washington is EVERYTHING Dying over...,"{'screen_name': 'SweetyPW', 'id': 35498686}",290620657828524032,2013-01-14 00:45:38
3,Anne Hathaway has got me living,"{'screen_name': '_NicoleEdwards', 'id': 144430...",290620657799159809,2013-01-14 00:45:38
4,Jennifer Lopezs lace dress Thoughts,"{'screen_name': 'lolaogunnaike', 'id': 134953223}",290620657778188288,2013-01-14 00:45:38


In [12]:
test = list(copy['text'].values)
print(len(test))
s = list(set(test))
print(len(s))
df = pd.DataFrame(s, columns=['text'])
df = df.loc[df['text'].str.contains('host')]
print(len(df))
df.head()

174643
123134
1551


,text
11,Id be more than happy to see Tina &amp; Amy ho...
75,RT : Dear folks: for future reference just le...
90,RT : “: This monologue is beyond incredible Ca...
93,I dedicate my 4k tweet to Tina Fey and Amy Poe...
249,RT : This show has hosts right


In [13]:
def findProperNouns(sen):
    tagged_sent = pos_tag(sen.split())
    propernouns = [word for word,pos in tagged_sent if pos == 'NNP']
    return propernouns
df['ProperNouns'] = df['text'].apply(lambda x: findProperNouns(x))
df.head(10)

,text,ProperNouns
11,Id be more than happy to see Tina &amp; Amy ho...,"[Tina, &amp;, Amy]"
75,RT : Dear folks: for future reference just le...,[SNL]
90,RT : “: This monologue is beyond incredible Ca...,"[Can, Tina, Fey, Amy, Poehler, ”]"
93,I dedicate my 4k tweet to Tina Fey and Amy Poe...,"[Tina, Fey, Amy, Poehler]"
249,RT : This show has hosts right,[]
295,Omg stop I havent laughed this much in forever...,[Omg]
313,Omg can they host together next year pls,[]
327,Can all award shows be hosted by people from SNL,[SNL]
358,One day well host too,[]
592,we would do this RT: : hosts Fey Poehler show...,"[RT:, Fey, Poehler]"


In [14]:
NGRAMS_DICT = {}
for i in range(len(df)):
    PN = df['ProperNouns'].iloc[i]
    if len(PN) % 2 == 0: 
        for j in range(0, len(PN), 2):
            name = PN[j] + ' ' + PN[j+1]
            if name in NGRAMS_DICT:
                NGRAMS_DICT[name] += 1
            else:
                NGRAMS_DICT[name] = 1

In [15]:
cd = sorted(NGRAMS_DICT.items(),key=operator.itemgetter(1),reverse=True)
cd

[('Tina Fey', 225),
 ('Amy Poehler', 205),
 ('Tina Amy', 99),
 ('Amy Tina', 59),
 ('Will Ferrell', 57),
 ('Kristen Wiig', 48),
 ('&amp; Amy', 19),
 ('Wiig Will', 19),
 ('Ferrell Oscars', 17),
 ('Fey Amy', 12),
 ('Amy Pohler', 10),
 ('Tina &amp;', 9),
 ('Kristin Wiig', 9),
 ('&amp; Tina', 7),
 ('Ricky Gervais', 6),
 ('Can Kristen', 6),
 ('Poehler &amp;', 6),
 ('Fey &amp;', 6),
 ('Anne Hathaway', 5),
 ('Fey Poehler', 5),
 ('Jay Leno', 5),
 ('Poehler Tina', 5),
 ('Ferrell Kristen', 5),
 ('Rudd Just', 5),
 ('Wiig Ferrell', 5),
 ('Golden G', 4),
 ('+ Amy', 4),
 ('Kristen Wig', 4),
 ('Please RT', 4),
 ('Amy &amp;', 4),
 ('Jennifer Lawrence', 4),
 ('Anyone Paul', 4),
 ('Paul Rudd', 3),
 ('Amy Oscars', 3),
 ('YES RT', 3),
 ('Fey Theyre', 3),
 ('Amy Poelher', 3),
 ('Im Tina', 3),
 ('Taylor Swift', 3),
 ('Kristen Wigg', 3),
 ('Fey y', 3),
 ('&amp; SAG', 2),
 ('Red Carpet', 2),
 ('Poehler Thank', 2),
 ('Bill Murray', 2),
 ('Amy Poehler:', 2),
 ('OMG Tina', 2),
 ('Ok Tina', 2),
 ('Golden Globe', 2

In [16]:
top3 = [i[0] for i in cd[0:2]]
top3

['Tina Fey', 'Amy Poehler']